### Relacja One to One

Aby zdefiniować relację One to One, tworzymy dwie klasy reprezentujące encję w bazie danych.


In [ ]:
@Entity
@Data
@AllArgsConstructor
@NoArgsConstructor
@Builder
public class Course {

    @Id
    @SequenceGenerator(
            name = "course_sequence",
            sequenceName = "course_sequence",
            allocationSize = 1
    )
    @GeneratedValue(
            strategy = GenerationType.SEQUENCE,
            generator = "course_sequence"
    )

    private Long courseId;
    private String title;
    private Integer credit;
}

####################################################################

@Entity
@Data
@AllArgsConstructor
@NoArgsConstructor
@Builder
public class CourseMaterial {

    @Id
    @SequenceGenerator(
            name = "course_material_sequence",
            sequenceName = "course_material_sequence",
            allocationSize = 1
    )
    @GeneratedValue(
            strategy = GenerationType.SEQUENCE,
            generator = "course_material_sequence"
    )

    private Long courseMaterialId;
    private String url;

    @OneToOne(
            cascade = CascadeType.ALL
    )
    @JoinColumn(
            name = "course_id",    ## create foreign key column named course_id
            referencedColumnName = "courseId"  ## refers to courseId in Course class

    )
    private Course course;
}

Następnie tworzymy dla obu klas repozytoria, a jeżeli chcemy przetestować działanie to tworzymy plik testowy dla CourseMaterial.

In [ ]:
@SpringBootTest
class CourseMaterialRepositoryTest {

    @Autowired
    private CourseMaterialRepository courseMaterialRepository;

    @Test
    public void SaveCourseMaterial() {

        Course course = Course.builder()
                .title("DSA")
                .credit(6)
                .build();

        CourseMaterial courseMaterial = CourseMaterial.builder()
                .url("www.google.com")
                .course(course)
                .build();

        courseMaterialRepository.save(courseMaterial);
    }
}

Niekiedy chcemy pobrać dane dla danej encji, bez pobierania danych powiązanych encji (np. chcemy pobrać nazwę firmy, ale bez pracujących tam pracowników itp.) Wówczas możemy zastosować dwa rodzaje "fetch-owania" - EAGERLY lub LAZY.

Różne relacje mają różne, domyślnie ustawione rodzaje fetchowania.

@OneToOne – FetchType.EAGER
@OneToMany – FetchType.LAZY
@ManyToOne – FetchType.EAGER
@ManyToMany – FetchType.LAZY

Jeżeli chcemy zastosować Fetch.LAZY dla relacji @OneToOne musimy dodać kod jak niżej

In [ ]:
@Entity
@Data
@AllArgsConstructor
@NoArgsConstructor
@Builder
@ToString(exclude = "course")    ## in tutorial this code was added, otherwise there was an error, but it worked for me without it
public class CourseMaterial {

    @Id
    @SequenceGenerator(
            name = "course_material_sequence",
            sequenceName = "course_material_sequence",
            allocationSize = 1
    )
    @GeneratedValue(
            strategy = GenerationType.SEQUENCE,
            generator = "course_material_sequence"
    )

    private Long courseMaterialId;
    private String url;

    @OneToOne(
            cascade = CascadeType.ALL,
            fetch = FetchType.LAZY,   ## here you set the fetch type
            optional = false ## it means that you can't add new course to DB without the related course material
    )
    @JoinColumn(
            name = "course_id",
            referencedColumnName = "courseId"

    )
    private Course course;
}


##### Relacja jedno lub dwukierunkowa

Na ten moment mamy ustawioną tylko relację jednokierunkową. W klasie encji CourseMaterial mamy ustawiony FOREIGN KEY dla atrybutu course. Aby zdefiniować relację dwukierunkową należy zmodyfikować klasę Course.

In [ ]:
@Entity
@Data
@AllArgsConstructor
@NoArgsConstructor
@Builder
public class Course {

    @Id
    @SequenceGenerator(
            name = "course_sequence",
            sequenceName = "course_sequence",
            allocationSize = 1
    )
    @GeneratedValue(
            strategy = GenerationType.SEQUENCE,
            generator = "course_sequence"
    )

    private Long courseId;
    private String title;
    private Integer credit;

    ## we have to add below code
    @OneToOne(
            mappedBy = "course",  ## refers to CourseMaterial's attribute which is set as a foreign key
            fetch = FetchType.LAZY
    )
    private CourseMaterial courseMaterial;
}